# Exercise 5 - Single Spike Train Analysis
Guy Singer
11.12.24

### Table of Contents

* [Part I: Spike Train Characteristics](#part1-spike-train)
   * [Last Week's Findings](#last-week)
   * [Interspike Interval Distribution](#interspike-interval)
   * [Poisson Spike Generator](#poisson-generator)
   * [Hazard and Survival Functions](#hazard-survival)

* [Part II: Cross- and Auto-correlation](#part2-correlation)
   * [Cross-correlation](#cross-correlation)
   * [Auto-correlation](#auto-correlation)
   * [Meaning of τ = 0 Value](#tau-zero)
   * [Predicting Neuron Type](#predict-neuron)
   * [Y-axis Values in Correlations](#y-axis)
   * [Non-stationarity](#non-stationarity)

## Part I: Spike Train Characteristics <a id='part1-spike-train'></a>

### Last Week's Findings <a id='last-week'></a>

Last week we deduced the expression for $P_T(n)$ or the model we use for the probability that any sequence of $n$ spikes occurs within a trial of duration $T$. After some inspection we came up with the following result:

$P_T(n) = \lim_{\Delta t \rightarrow 0} \binom{M}{n} (r \Delta t)^n (1 - r\Delta t)^{M - n}$

which can be simplified to

$P_T(n) = \frac{(rT)^n}{n!}e^{-rT}$

which also describes a Poisson process with a parameter of $rT$. From $P_T(n)$ we can deduce the chance for a *specific* spike train, i.e. a train with spikes at given times $\{t_1, \ldots, t_n\}$:

$P(t_1,\ldots,t_n) = n! P_T(n) \left( \frac{\Delta t}{T} \right)^n$

### Interspike Interval Distribution <a id='interspike-interval'></a>

A useful metric for spike trains in their interspike interval, i.e. the time between adjacent spikes. This statistical property is a way for us to discriminate between different neuronal behaviors in a quantitative manner. To arrive at that distribution, we assume that a spike occurred at time $t_i$ and we ask ourselves what is the probability (in an HPP) that another spike will occur in the short interval $t_i + \tau \leq t_{i+1} < t_i + \tau + \Delta t$ (figure 1, assuming $\Delta t$ is very small). That is, what is the chance for a spike $\tau$ seconds after the previous, with an interval of $\Delta t$ after it.

![figure](isi.png)

*Figure 1: Spike timing definitions.*

To answer, we need to see what is the probability that no spikes will fire for $\tau$ seconds, and then follow up with the probability that a spike will occur in the subsequent $\Delta t$ interval. The chance for $n=0$ spikes in the first interval is:

$P_\tau(n=0) = \frac{(r\tau)^0}{0!}e^{-r\tau} = e^{-r\tau}$

and the chance for a spike in a short interval $\Delta t$ is by definition $r \Delta t$. Thus, the probability of an interspike interval with a value of $\tau$ is:

$P(t_i + \tau \leq t_{i+1} < t_i + \tau + \Delta t) = r\Delta t e^{-rT}$

The PDF of interspike intervals is the same expression as above without the $\Delta t$, making it an exponential probability density function. From this inter-spike interval one can create the time interval histogram (TIH), which has an exponential trend, as seen in figure 2. These quantities, the ISI and the TIH indicate that long ISIs are generally less likely, at least in the case of the simple Poisson neuron. Any divergence from a simple exponential may indicate of some underlying property that will probably be of interest. For example, an oscillatory neuron will have a peak in its TIH at that expected time interval.

![figure](tih.png)

*Figure 2: Time interval histogram. X-axis is time in ms, Y-axis is percentage of total spikes. From Izhar Bar-Gad lectures.*

One notable difference we already see between a pure exponential and the TIH depicted above, is the number of ISIs in the lower-valued bins. Instead of the TIH obeying a simple exponential decay, it starts off very low, and only after about 10 ms it starts the expected decay process. This is due to the refractory period of real neurons, that was not accounted for during our derivation process. A more accurate function that describes the measured distribution is the Gamma function.

### Question
What is the expected mean interspike interval?

### Answer
We'll compute the expected value of the PDF to find the mean interval:

$\begin{aligned}
\left<\tau\right> & = \int_0^\infty \tau p(\tau) d\tau \\
& = \int_0^\infty \tau r e^{-r\tau} d\tau \\
& = r \int_0^\infty \tau e^{-r\tau} d \tau \text{\ \ \ (Gamma function)}\\
& = r \left[ \frac{1}{r^2} \right] = \frac{1}{r}
\end{aligned}$

This result is not surprising - for an average firing rate of $r$, the mean interval between spikes is $\frac{1}{r}$.

### Poisson Spike Generator <a id='poisson-generator'></a>

With the model of a Poisson process for a spike train, we can use $r(t)$ to simulate the firing of a neuron, by "abusing" the fact that the probability of firing a spike in a short interval of time $\Delta t$ is $r\Delta t$. To generate the spikes we create a vector with random numbers of the same length of our time series, and check whether the random number in that bin each greater than the $r(t)$ for that bin. If that is the case we determine that a spike occurred in that bin.

An important deviation of real neurons from a straight-forward Poisson process is their refractory period, as we mentioned during our discussion of the TIH. To account for it, we can very easily prohibit spikes in a time interval following a generated spikes. The resulting TIH from the generated data should closely resemble a "true" TIH as seen in figure 2.

Yet another real-world deviation from the Poisson process are bursting neurons. Burst activity is a very important feature of some neurons, and a simple spike generator fails to mimic it. Two general solution exist - either model the interval between bursts as a Poisson process, while modeling the spikes within the bursts themselves in a different manner, or increase the probability of firing after a spike by some factor and re-conduct the chance-to-spike calculation during that part.

### Hazard and Survival Functions <a id='hazard-survival'></a>

Integration of the probability density of the interspike intervals function yields probability. For example:

$P(\text{neuron fires until }t) = \int_{\hat{t}}^t r e^{-r(t'-\hat{t})}dt'$

where $\hat{t}$ is the time the neuron last fired. The probability the neuron *doesn't fire* in that period is called the *survival function*, and is just $S(t|\hat{t}) = 1 - P(\text{neuron fires until }t)$. 

The nickname was given to emphasize that the neuron has to "survive" until the end of the period without spiking. The initial value of the function is 1, and it decreases to 0 as $t \rightarrow \infty$. It can also be thought of as a 1 - (cumulitive sum of the time-interval histogram). The rate in which it decays is called the *hazard function*, and is written as:

$\rho(t|\hat{t}) = -\frac{d(S(t|\hat{t}))}{dt} \frac{1}{S(t|\hat{t})} = -\frac{P(\text{neuron fires until }t)}{S(t|\hat{t})}$

The hazard function can be thought of as the probability to spike at some given point given that it spiked before. After long ISIs it becomes noisy, since there are very few neurons that spike with such a long ISI, which influences the result hazard function value greatly.

## Part II: Cross- and Auto-correlation <a id='part2-correlation'></a>

### Cross-correlation <a id='cross-correlation'></a>

Given two signals, we can ask ourselves how similar they are. When calculating the correlation we look at the two signals in a specific time period or "window" and compare them. The mathematical definition is similar to convolution:

$(f \star g)(\tau) \equiv \sum_{n=-\infty}^\infty f[n] g[n+\tau]$

Intuitively, we go through every value in the system, multiplying both functions, after sliding one of the functions by a small step. In places where their values are both high throughout the spectrum, the multiplication will result in a higher value, creating a peak in the cross-correlation plot. Cross-correlation can identify time-shifting of functions easily, as for a certain value of $n$ we'll find a peak in the cross-correlation value.

![figure](xcorr.png)

*Figure 3: Signal of two sine waves (left) with their cross correlation. Notice the peak at $t=5$ time units.*

The demonstration shows two sine waves with the same frequency, one of them having been time-shifted by $\pi$ radians. The periodicity of 10 time units was preserved in the cross-correlation plot as well, and its peak at 5 time units indicates that one of the signals has been shifted by half a period from the other (hence the 5 time units difference). When dealing with neural data, the fact that cross-correlation histogram has a peak some distance away from the origin indicates a phase-locked response between both neurons.

### Question

Calculate the cross-correlation of the following two signals:

$f[n] = [-1 \  4 \ 2] \ ; \ g[n] = [8 \ -3 \ 4]$

### Answer

We'll do it step-by-step. First we consider the simple fact that the value of cross-correlation is different for each delay $\tau$. This means that the cross-correlation of two signals is a function by itself, and we could ask what is its value for $\tau = 0$, or $\tau = 2$. Here, though, we're interested in its values for *all* $\tau$'s.

Our method of operation will be as follows - we'll take an offset (=$\tau$), multiply our functions, sum them up - and move on to the next value of $\tau$. It's evident that if we offset $x_2[n]$ by too much, either to the left or to the right - the value of the cross-correlation function will be *zero*. That is because there's no overlap between the two signals when they're that far apart. For example - if we look at an offset value of $\tau = 3$:

```
    -1  4  2
*            8 -3  4
0   0  0  0  0  0  0  ⇒ (f ⋆ g)(τ = ±3) = 0
```

So it's clear to us that for offsets greater or equal in their absolute value than three - the result of the cross-correlation will just be 0. Now we'll examine each value of $\tau$ and calculate the cross-correlation:

For $\tau = 2$:
```
    -1  4  2
*          8 -3  4
0    0  0 16  0  0  0  ⇒ (f ⋆ g)(τ = 2) = 16
```

For $\tau = 1$:
```
    -1  4  2
*       8 -3  4
0    0 32 -6  0  0  0  ⇒ (f ⋆ g)(τ = 1) = 26
```

For $\tau = 0$:
```
    -1  4  2
*    8 -3  4
0   -8 -12 8  0  0  0  ⇒ (f ⋆ g)(τ = 0) = -12
```

For $\tau = -1$:
```
      -1  4  2
*   8 -3  4
0   0  3 16  0  0  0  ⇒ (f ⋆ g)(τ = -1) = 19
```

For $\tau = -2$:
```
        -1  4  2
*  8 -3  4
0  0  0 -4  0  0  0  ⇒ (f ⋆ g)(τ = -2) = -4
```

And thus, the cross-correlated signal is:

$(x_1 \star x_2)(\tau) = [-4 \ \ 19 \ \ -12 \ \ 26 \ \ 16]$

while in the rest of the possible $\tau$ values it's nullified. Cross-correlation (and auto-correlation) will always have $2N - 1$ values, if $N$ is the number of samples in each train.

### Question

How should the cross-correlogram of the ISIs of two unrelated neurons look like?

### Answer

If neuron A is completely independent of neuron B, then it's clear that if neuron A is the reference neuron, and we measure the time delays between A's spikes and B spikes (thus creating the cross-correlogram), these time delays should be spread evenly across all possible times. Remember that we measure the time delay between a spike and all preceding and following spikes in the other train. And so it follows that the correlogram will be completely flat across the entire $t$-axis, indicating of no specific correlation in any point in time. Mathematically speaking, its integral over any section will be zero.

### Auto-correlation <a id='auto-correlation'></a>

Auto-correlation is the correlation of a signal with its past and future self. When it's positive, it intuitively indicates that a signal is *'persistent'*, remaining in the same state from one time-point to another. The obvious example is rain - when it's raining today, it's more likely to rain tomorrow. If we have a time series describing the amount of rain pouring every day, it's more likely that Monday will be rainy if Sunday was rainy as well.

![figure](Comparison_convolution_correlation.png)

*Figure 4: Comparison of convolution, cross-correlation and auto-correlation. From Wikipedia.*

The mathematical expression for auto-correlating a discrete signal $x[n]$ is:

$(f \star g)(\tau) \equiv \sum_{n = - \infty}^{\infty}{f[n] g[n + \tau]}$

Properties:
* Auto-correlation is an even function: $R_{xx}(t) = R_{xx}(-t)$
* The period of the auto-correlation function is similar to the period of the original function.
* The auto-correlation of a pure noise signal will be a Dirac delta function.

### Question

Compute the auto-correlation function of the following discrete signal:

$x = [1, 3, -4]$

### Answer

First we recognize that wherever the signal is not defined, we can treat it as a 0. The multiplication of a signal with itself is just like regular multiplication, but when summing them all up we'll just translate one of the signals away from the origin.

```
    1  3  -4
*   1  3  -4
4  -9  26  -9  -4
```

As expected the signal is indeed even (figure 5). We could also calculate just half of the numbers, using this evenness property. Figure 6 shows real auto-correlation data taken from an article. The oscillatory pattern of the firing of the neuron is clearly visible. The trough in the center of the plot indicates the refractory period of the neuron.

![figure](autocorr1.png)

*Figure 5: Matlab simulation of the generated auto-correlation function.*

![figure](autocorr.png)

*Figure 6: Real neuron data showing auto-correlation. The center peak was dropped. From Bar-Gad et al., JoN Meth. 2001.*

### Meaning of the τ = 0 value for auto-correlation <a id='tau-zero'></a>

From the example above it's clear why does the auto-correlated signal is always symmetric in regards to $\tau = 0$. Let's consider the value at $\tau = 0$. As we could see, for $\tau = 0$ the cross-correlation value is just the sum of multiplication of all signal values one by the other. But what would this value represent if our signal would've been a *spike train*, in which each bin represents a time stamp of a spike, and the bin's value can only be either 1 or 0? Moreover, what would this value represent if we're calculating the auto-correlation of a signal, rather than its cross-correlation with a different signal?

### Predicting the type of neuron based on its auto-correlation <a id='predict-neuron'></a>

What type of neuron is depicted in figure 7?

![figure](burster.png)

*Figure 7: Understanding a neuron's properties from its auto-correlation. A peak in the lower-valued time bins means that after a spike there's a bigger chance the neuron will spike again, hinting towards a bursty behavior.*

The graph indicates that for very small offsets there's a high correlation of the signal. So "moving" the signal by a few timebins and calculating the correlation doesn't affect the result much. This indicates that there are certain periods of time in which the neuron fires a lot -- a bursty neuron.

Let's look at another one, in figure 8. From the graph it seems that if we move the whole spike train by some $\tau = T$ timebins - there's a higher correlation between the signals. But we can also move the signals away from each other by $\tau = 2T$ and still receive approximately the same result, indicating that this neuron fires with a period corresponding to $\tau = T$.

![figure](osci.png)

*Figure 8: This autocorrelation shows a recurring peak in a fixed interval, which means that the neuron's ISI is approximately constant and that the neuron itself exhibits an oscillating behavior.*

### Y-axis values in correlations <a id='y-axis'></a>

One more issue with the auto-correlation is its absolute values. In statistics this function is limited between $[-1, 1]$, but we've already seen how we usually receive values much higher than that. A way to generate auto-correlation values more like their statistical counterparts is through normalization and shuffle-correction.

We already know that the $\tau = 0$ term in the center of the auto-correlation plot (sometime also called *auto-correlogram*) is the sum of all spikes in a train. A very simple normalization act would to just divide the whole correlogram by this value. The issue with this solution has to do with the fact that this center value is usually several orders of magnitude larger than all other values in the correlogram, and thus it's usually just taken out of the plot.

Another approach, which is not mutually-exclusive with the first, is to subtract the mean firing rate from the correlation plot:

$(f \star g)(\tau)_{\text{unbiased}} = \sum_{n = - \infty}^{\infty}{(f[n] - \langle r \rangle)(g[n - \tau] - \langle r \rangle)}$

where $\langle r \rangle$ is the trial-averaged spike rate: $\langle r \rangle = \frac{\langle N \rangle}{T}$ and $\langle N \rangle$ is the average number of spikes in all trials, while $T$ is the total time of a trial.

Those of you who know some statistics might call it an auto-covariance function. Subtracting the mean makes it easier for us to see when the correlation is negative (anti-correlated) or positive.

Normalization of the cross-correlation correlogram can be done using a variety of methods. When we're dealing with spike trains that contain only zeros and ones, a common method is to divide the result by the total number of bins summed times the total amount of spike, to receive probability:

$(f \star g)(\tau)_{\text{normed}} = \frac{1}{T \cdot N} \sum_{n = -\frac{T}{2}}^{\frac{T}{2}}{f[n] g[n + \tau]}$

This is the probability since the total number of counts we have is that in every bin we'll have maximal correlation, which is just the number of spikes. Multiplying them both is thus probability = 1. In other applications, such as processing correlation algorithms, one would also subtract the mean from both signals.

### Non-stationarity <a id='non-stationarity'></a>

When correlating over long periods of time were assuming *stationarity*, meaning that the recorded neuron's internal state hasn't change during the recording. A change in its internal state, which would result in a change in the Possion coefficient of its firing rate ($\lambda$), might turn the result from the correlation invalid.

There's no general way to correct for this pitfall, unfortunately. We should at least be aware of it, and try to detect it using analysis tools like a *raster plot*, which will be discussed in the next recitation. A raster plot will give us an over-arching view of the recorded signal from all neurons, and might make it easier to detect tactonic shift in the state of the recorded neurons.